In [ ]:
import pandas as pd
import numpy as np


: 

In [ ]:
books_df = pd.read_csv('books_data/books.csv',encoding='ISO-8859-1', sep=';', on_bad_lines='skip')
ratings_df = pd.read_csv('books_data/ratings.csv',encoding='ISO-8859-1', sep=';')
users_df = pd.read_csv('books_data/users.csv',encoding='ISO-8859-1', sep=';')

In [ ]:
books_df.head()

In [ ]:
ratings_df.head()

In [ ]:
users_df.head()

In [6]:
cluster_df = ratings_df.merge(users_df,on='User-ID').merge(books_df[['ISBN','Year-Of-Publication','Book-Author']],on='ISBN').dropna().reset_index().drop(columns=['index'])

In [ ]:
cluster_df.head()

In [8]:
from sklearn.preprocessing import LabelEncoder


In [9]:
cat_cols = ['ISBN', 'Location','Book-Author','Year-Of-Publication']

In [10]:
les = {}
cluster_df = cluster_df[~cluster_df['Year-Of-Publication'].isin(['DK Publishing Inc','Gallimard'])].reset_index().drop(columns=['index'])
cluster_df['Year-Of-Publication'] = cluster_df['Year-Of-Publication'].map(str)
df = pd.DataFrame(cluster_df)
for col in cat_cols:
    le = LabelEncoder()
    les[col] = le
    df[col] = le.fit_transform(cluster_df[col])


In [ ]:
df.head()

In [12]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


In [13]:
scaler = StandardScaler().fit(df)
s_df = scaler.transform(df)

In [14]:
from sklearn.cluster import KMeans
# load data
inertias = []
MAX_CLUSTERS = 34
K = range(11,MAX_CLUSTERS,5)

In [ ]:


# try different numbers of clusters
for n_clusters in K:
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(s_df)
    labels = kmeans.predict(s_df)
    print("Cluster Centroids for {} clusters:".format(n_clusters))
    print(kmeans.cluster_centers_)
    print("\nCluster Labels for {} clusters:".format(n_clusters))
    print(labels)
    inertias.append(kmeans.inertia_)
plt.figure()
plt.plot(K,inertias, label='Loss')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [16]:
N_CLUSTERS = 26

kmeans = KMeans(n_clusters=25)
kmeans.fit(s_df)
labels = kmeans.predict(s_df)

In [17]:
cluster_df['Label'] = labels
df['Label'] = labels


In [ ]:
cluster_df.head()

In [19]:
import xgboost as xgb
import sklearn.metrics as metrics


In [78]:
x_cols = ['User-ID','ISBN','Age','Year-Of-Publication','Book-Author','Label']
y_cols = ['Book-Rating']
msk = np.random.rand(len(df)) < .8

X_train = df[msk][x_cols]
y_train = df[msk][y_cols]

X_test = df[~msk][x_cols]
y_test = df[~msk][y_cols]


In [79]:
def train_test_xgb(X_train,y_train,X_test,y_test):
    xgb_model = xgb.XGBClassifier(objective='multi:softmax', random_state=42)
    xgb_model.fit(X_train, y_train)
    
    y_pred = xgb_model.predict(X_test)
    acc = metrics.accuracy_score(y_test, y_pred)
    dist = metrics.mean_absolute_error(y_test,y_pred)
    xgb_model.save_model('xgboost_model'+str(acc)+'.json')
    print('XGB Accuracy: %.9f%%' % (acc*100))
    print('Mean absoulte error: ', dist)
    return acc

In [ ]:
train_test_xgb(X_train,y_train,X_test,y_test)

In [ ]:
cluster_df.groupby('User-ID').count().reset_index().sort_values(by='ISBN')['ISBN'].quantile(.8)

In [ ]:
ratings_df.head()